In [1]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split

os.listdir()

['Mexico_data_preprocessing.ipynb', 'RSNA_data_preprocessing.ipynb']

In [2]:
os.chdir("../")
main_dir = os.getcwd()
main_dir

'D:\\David Mexico\\Bone age estimation\\bone_age_estimation'

In [3]:
age_df = pd.read_csv("data/rsna-bone-age/training/train.csv")
age_df.head()

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False


In [4]:
# Map id to path
age_df['path'] = age_df['id'].map(lambda x: os.path.join(main_dir,
                                                         'data',
                                                         'rsna-bone-age',
                                                         'training',
                                                         'boneage-training-dataset',
                                                         '{}.png'.format(x)))

In [5]:
# Checking if all the images exist
age_df['exists'] = age_df['path'].map(os.path.exists)
print(age_df['exists'].sum(), 'images found of', age_df.shape[0], 'total')

# Convert boolean male values to gender 0(male) and 1(female)
age_df['gender'] = age_df['male'].map(lambda x: 0 if x else 1)

12611 images found of 12611 total


In [6]:
boneage_mean = age_df['boneage'].mean()
boneage_div = 2 * age_df['boneage'].std()
# we don't want normalization for now
boneage_mean = 0
boneage_div = 1.0
age_df['boneage_zscore'] = age_df['boneage'].map(lambda x: (x - boneage_mean) / boneage_div)
age_df.dropna(inplace=True)

# Creating bins for the boneage
age_df['boneage_category'] = pd.cut(age_df['boneage'], 10)
age_df.sample(3)

,id,boneage,male,path,exists,gender,boneage_zscore,boneage_category
5047,7166,132,False,D:\David Mexico\Bone age estimation\bone_age_e...,True,1,132.0,"(114.5, 137.2]"
9303,11920,150,True,D:\David Mexico\Bone age estimation\bone_age_e...,True,0,150.0,"(137.2, 159.9]"
2220,3847,144,False,D:\David Mexico\Bone age estimation\bone_age_e...,True,1,144.0,"(137.2, 159.9]"


In [7]:
# Splitting the data into train and validation based on the boneage_category
age_df['boneage_category'] = pd.cut(age_df['boneage'], 10)
raw_train_df, valid_df = train_test_split(age_df,
                                          test_size=0.25,
                                          random_state=2018,
                                          stratify=age_df['boneage_category'])
# Saving the dataframe with boneage, path, id and gender information
train_df = raw_train_df[['id', 'boneage', 'path', 'gender']]
validation_df = valid_df[['id', 'boneage', 'path', 'gender']]

In [8]:
# save dataframe not index
train_df.to_csv('data/rsna-bone-age/training/train_df.csv', index=False)
validation_df.to_csv('data/rsna-bone-age/training/valid_df.csv', index=False)

In [17]:
592*16

9472